In [6]:
import pandas as pd
import numpy as np
import ast
from sklearn.svm import SVC
import wfdb
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer

In [2]:
from utils import utils
import pickle

In [33]:
import torch
torch.cuda.is_available()

False

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

from fastai.layers import *
from fastai.core import *

In [7]:
def load_raw_data(df, sampling_rate, path, num):
    if sampling_rate == 100:
        data = [wfdb.rdsamp(path+f) for f in df.filename_lr]
    else:
        data = list()
        start = int(num * (len(df) / 4))
        end = int((num + 1)* (len(df) / 4))
        it = start
        for f in df.filename_hr[start:]:
            if num != 3:
                if it >= end:
                    break
            data.append(wfdb.rdsamp(path+f))
            it += 1
        #data = [wfdb.rdsamp(path+f) for f in df.filename_hr]
    data = np.array([signal for signal, meta in data])
    return data

In [8]:
def load_dataset(path, sampling_rate, num):
    # load and convert annotation data
    Y = pd.read_csv(path+'ptbxl_database.csv', index_col='ecg_id')
    Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))
    Y['scp_codes_len'] = Y.scp_codes.apply(lambda x: len(x))

    # Load raw signal data
    X = load_raw_data(Y, sampling_rate, path, num)
    return X, Y

In [9]:
def aggregate_diagnostic(y_dic):
    tmp = []
    for key in y_dic.keys():
        if key in aggregation_df.index:
            tmp.append(aggregation_df.loc[key].diagnostic_class)
    return list(set(tmp))

In [22]:
def select_data(X, Y, min_samples, outputfolder):
    # convert multilabel to multi-hot
    mlb = MultiLabelBinarizer()
    min_samples=0
    counts = pd.Series(np.concatenate(Y.superdiagnostic.values)).value_counts()
    counts = counts[counts > min_samples]
    Y.superdiagnostic = Y.superdiagnostic.apply(lambda x: list(set(x).intersection(set(counts.index.values))))
    Y['superdiagnostic_len'] = Y.superdiagnostic.apply(lambda x: len(x))
    X = X[Y.superdiagnostic_len > 0]
    Y = Y[Y.superdiagnostic_len > 0]
    
    X = X[Y['burst_noise'].isna()]
    Y = Y[Y['burst_noise'].isna()]
    
    X = X[Y['electrodes_problems'].isna()]
    Y = Y[Y['electrodes_problems'].isna()]
    
    mlb.fit(Y.superdiagnostic.values)
    
    
    Y_new = mlb.transform(Y.superdiagnostic.values)
    with open(outputfolder+'mlb.pkl', 'wb') as tokenizer:
        pickle.dump(mlb, tokenizer)
        
    return X, Y, Y_new


In [16]:
X_noisy.shape

(21224, 1000, 12)

In [17]:
X.shape

(21837, 1000, 12)

In [11]:
sampling_rate = 100
num = 0
path = '../dataset-1/'
X, Y = load_dataset(path, sampling_rate, num)

In [12]:
# Load scp_statements.csv for diagnostic aggregation
Y['scp_codes_len'] = Y.scp_codes.apply(lambda x: len(x))
aggregation_df = pd.read_csv(path + 'scp_statements.csv', index_col=0)
aggregation_df = aggregation_df[aggregation_df.diagnostic == 1.0]

# Apply diagnostic superclass
Y['superdiagnostic'] = Y.scp_codes.apply(aggregate_diagnostic)
Y['superdiagnostic_len'] = Y.superdiagnostic.apply(lambda x: len(x))

In [24]:
output_folder = 'output/'
X, Y_lbl, Y = select_data(X, Y, min_samples=0, outputfolder=output_folder)

In [26]:
with open("output/X_noise.csv", "wb") as fp:   #Pickling
    pickle.dump(X, fp)

In [27]:
with open("output/Y_noise.csv", "wb") as fp:   # Unpickling
    pickle.dump(Y, fp)

In [28]:
with open("output/Y_lbl_noise.csv", "wb") as fp:   # Unpickling
    pickle.dump(Y_lbl, fp)

In [16]:
X.shape, Y.shape, Y_lbl.shape

((20849, 1000, 12), (20849, 5), (20849, 30))

In [28]:
standard_scaler = pickle.load(open('output/'+experiment+'/data/standard_scaler.pkl', "rb"))

X_train = utils.apply_standardizer(X_train, standard_scaler)
X_test = utils.apply_standardizer(X_test, standard_scaler)

In [86]:
a = [0, 1, 2, 3]
with open("output/test1.csv", "ab") as fp: 
    pickle.dump(a, fp)

In [87]:
b = [[1, 2, 3, 4]
with open("output/test2.csv", "ab", ) as fp: 
    pickle.dump(b, fp)

In [88]:
with open("output/test1.csv", "rb") as fp:   # Unpickling
    a = pickle.load(fp)
with open("output/test2.csv", "rb") as fp:   # Unpickling
    b = pickle.load(fp)


In [111]:
a = np.array([[[1, 2, 3, 4], [1, 2, 3, 4], [1, 2, 3, 4]], [[1, 2, 3, 4], [1, 2, 3, 4], [1, 2, 3, 4]]])
b = np.array([[[0, 1, 2, 3], [0, 1, 2, 3], [0, 1, 2, 3]]])
c = list(a)
c.extend(list(b))
np.array(c).shape

(3, 3, 4)

In [45]:
X

array([[[-1.190e-01, -5.500e-02,  6.400e-02,  8.600e-02, ...,  0.000e+00, -2.600e-02, -3.900e-02, -7.900e-02],
        [-1.160e-01, -5.100e-02,  6.500e-02,  8.300e-02, ..., -3.000e-03, -3.100e-02, -3.400e-02, -7.400e-02],
        [-1.200e-01, -4.400e-02,  7.600e-02,  8.200e-02, ..., -1.000e-02, -2.800e-02, -2.900e-02, -6.900e-02],
        [-1.170e-01, -3.800e-02,  8.000e-02,  7.700e-02, ..., -1.500e-02, -2.300e-02, -2.200e-02, -6.400e-02],
        ...,
        [ 9.000e-02,  2.100e-02, -6.900e-02, -5.500e-02, ..., -2.300e-02, -2.700e-02, -3.600e-02, -8.000e-03],
        [ 6.900e-02,  0.000e+00, -6.900e-02, -3.400e-02, ...,  0.000e+00,  2.400e-02, -4.100e-02, -5.800e-02],
        [ 8.600e-02,  4.000e-03, -8.100e-02, -4.400e-02, ..., -2.500e-02,  2.420e-01, -4.600e-02, -9.800e-02],
        [ 2.200e-02, -3.100e-02, -5.400e-02,  5.000e-03, ..., -1.490e-01,  1.430e-01, -3.500e-02, -1.200e-01]],

       [[ 4.000e-03,  1.380e-01,  1.340e-01, -7.200e-02, ...,  1.430e-01,  1.920e-01,  8.300e-02,

In [15]:
# Split data into train and test
test_fold = 10
# Train
X_train = X[np.where(Y_lbl.strat_fold != test_fold)]
y_train = Y[(Y_lbl.strat_fold != test_fold)]
# Test
X_test = X[np.where(Y_lbl.strat_fold == test_fold)]
y_test = Y[Y_lbl.strat_fold == test_fold]